In [1]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from facenet_pytorch import MTCNN
import os, cv2
import albumentations as A

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
mtcnn = MTCNN(keep_all=True, device=device)
new_img_dir = '/opt/ml/input/data/train/new_imgs'
img_path = '/opt/ml/input/data/train/images'
if not os.path.exists(new_img_dir):
            os.makedirs(new_img_dir)

cnt = 0
detected = 0

for paths in os.listdir(img_path):
    if paths[0] == '.': continue
    
    sub_dir = os.path.join(img_path, paths)

    for imgs in os.listdir(sub_dir):

        if imgs[0] == '.': continue
        
        img_dir = os.path.join(sub_dir, imgs)
        img = cv2.imread(img_dir)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        
        #mtcnn 적용
        boxes,probs = mtcnn.detect(img)
        if len(probs) > 1: 
            detected += 1
            #print(boxes)
        # no face found
        if not isinstance(boxes, np.ndarray):
            # 직접 crop
            #img=img[100:400, 50:350, :]
            img = A.CenterCrop(height = 384, width = 384)(image = img)['image']

        # boexes size 확인
        else:
            xmin = int(boxes[0, 0])-30
            ymin = int(boxes[0, 1])-30
            xmax = int(boxes[0, 2])+30
            ymax = int(boxes[0, 3])+30
            
            if xmin < 0: xmin = 0
            if ymin < 0: ymin = 0
            if xmax > 384: xmax = 384
            if ymax > 512: ymax = 512
            
            img = img[ymin:ymax, xmin:xmax, :]
            
        tmp = os.path.join(new_img_dir, paths)
        if not os.path.exists(tmp):
            os.makedirs(tmp)

        file_name = os.path.join(tmp,imgs)
        #print(file_name)
        
        plt.imsave(file_name, img)
        cnt += 1
        print(detected, cnt)

#668 18900

In [ ]:
import cv2

src = cv2.imread('/opt/ml/input/data/train/images/000001_female_Asian_45/normal.jpg')
classifier = cv2.CascadeClassifier('haarcascade_frontalface_alt2.xml')

faces = classifier.detectMultiScale(src) # 스케일팩터를 1.2로 지정해도 잘 작동함 더 빨라짐

# 각각의 행마다 (x,y,w,h) 받아와서 사각형을 그리는 코드
#for (x, y, w, h) in faces:
#    cv2.rectangle(src, (x, y, w, h), (255, 0, 255), 2)

print(faces)

for (x,y,w,h) in faces:
    cv2.rectangle(src, (x, y, w, h), (255, 0, 255), 2)
cv2.imshow('src', src)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
import cv2

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
new_img_dir = '/opt/ml/input/data/train/new_imgs'
img_path = '/opt/ml/input/data/train/images'
if not os.path.exists(new_img_dir):
            os.makedirs(new_img_dir)

cnt = 0
detected = 0
classifier = cv2.CascadeClassifier('haarcascade_frontalface_alt2.xml')

for paths in os.listdir(img_path):
    if paths[0] == '.': continue
    
    sub_dir = os.path.join(img_path, paths)

    for imgs in os.listdir(sub_dir):

        if imgs[0] == '.': continue
        
        img_dir = os.path.join(sub_dir, imgs)
        img = cv2.imread(img_dir)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        
        
        #mtcnn 적용
        faces = classifier.detectMultiScale(img)
        if len(faces)>=1: 
            detected += 1
         
        cnt += 1
        print(detected, cnt)